In [21]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor,
    RANSACRegressor, TheilSenRegressor, BayesianRidge, SGDRegressor
)
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from tqdm import tqdm

from utils import create_data_loaders, train_model, test_model

#### Hyperparameters

In [2]:
RANDOM_STATE = 42
BATCH_SIZE = 32
NUM_EPOCHS = 40
LEARNING_RATE = 1e-3
PATIENCE = 10
MIN_DELTA = 1e-4

In [3]:
run_matrices = load('run_matrices.joblib')
incoming_run_matrices = load('incoming_run_matrices.joblib')
metrology_matrix = load('metrology_matrix.joblib')

In [4]:
print(run_matrices.shape, incoming_run_matrices.shape, metrology_matrix.shape)
print(type(run_matrices), type(incoming_run_matrices), type(metrology_matrix))

(4140, 755, 20) (4140, 755, 45) (4140, 49)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Train, val, test split

In [12]:
X = np.concatenate([run_matrices, incoming_run_matrices], axis=2)
y = metrology_matrix

# Convert to torch tensors
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)

# Define dataset sizes
total_size = len(X)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

# Create TensorDataset to keep X and y aligned
dataset = torch.utils.data.TensorDataset(X, y)

# Split dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(RANDOM_STATE)
)

# Extract X and y from datasets
X_train, y_train = next(iter(torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset))))
X_val, y_val = next(iter(torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset))))
X_test, y_test = next(iter(torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))))


### Flattening

In [13]:
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_val_flattened = X_val.reshape(X_val.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)
print(X_train_flattened.shape)

torch.Size([3312, 49075])


### Linear Models

In [22]:
models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ]),
    # "Lasso": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", Lasso(alpha=0.1))
    # ]),
    # "ElasticNet": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", ElasticNet(alpha=0.1, l1_ratio=0.5))
    # ]),
    # "HuberRegressor": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", MultiOutputRegressor(HuberRegressor()))
    # ]),
    # "RANSACRegressor": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", MultiOutputRegressor(RANSACRegressor(min_samples=10)))
    # ]),
    # "TheilSenRegressor": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", MultiOutputRegressor(TheilSenRegressor()))
    # ]),
    # "BayesianRidge": Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("model", MultiOutputRegressor(BayesianRidge()))
    # ]),
    "SGDRegressor": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MultiOutputRegressor(SGDRegressor(max_iter=1000, tol=1e-3)))
    ])
}

In [ ]:
for model_name, pipeline in models.items():
    pipeline.fit(X_train_flattened, y_train)
    # scaled_sample = pipeline.named_steps['scaler'].transform(X_test_flattened)[0]
    # print("First scaled sample:", scaled_sample)
    y_test_pred = pipeline.predict(X_test_flattened)
    print(f"{model_name}:")
    print(f"mse: {mean_squared_error(y_test_pred, y_test)}")
    print(f"r2_score: {r2_score(y_test_pred, y_test)}")
